In [1]:
import pandas as pd
import sqlite3
import random
from contextlib import contextmanager

# Step 1: Generate sample weather data (similar to previous request)
def generate_sample_data(num_records=1000):
    years = list(range(1900, 2021))
    data = {
        'record_id': range(1, num_records + 1),
        'year': [random.choice(years) for _ in range(num_records)],
        'temperature_c': [random.uniform(-50, 50) for _ in range(num_records)]
    }
    return pd.DataFrame(data)

# Step 2: Simulate Hive database and table in SQLite
@contextmanager
def sqlite_connection(db_name):
    conn = sqlite3.connect(db_name)
    try:
        yield conn
    finally:
        conn.close()

def setup_hive_like_db():
    # Create a SQLite database (simulating Hive database)
    db_name = 'weather_hive.db'
    df = generate_sample_data(1000)

    with sqlite_connection(db_name) as conn:
        # Create table (simulating Hive CREATE TABLE)
        df.to_sql('weather_data', conn, if_exists='replace', index=False)

        # Create index on year (simulating Hive CREATE INDEX)
        conn.execute('CREATE INDEX idx_year ON weather_data(year)')

        # Create view (simulating Hive CREATE VIEW)
        conn.execute('''
            CREATE VIEW positive_temps AS
            SELECT record_id, year, temperature_c
            FROM weather_data
            WHERE temperature_c > 0
        ''')

    print(f"Database '{db_name}', table 'weather_data', index 'idx_year', and view 'positive_temps' created.")

# Step 3: User-Defined Function (UDF) to convert Celsius to Fahrenheit
def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9/5) + 32

# Register UDF in SQLite
def register_udf(conn):
    conn.create_function('c_to_f', 1, celsius_to_fahrenheit)
    print("UDF 'c_to_f' registered.")

# Step 4: Generate weather report (min/max temps per year)
def generate_weather_report():
    db_name = 'weather_hive.db'
    with sqlite_connection(db_name) as conn:
        # Register UDF
        register_udf(conn)

        # Query table for min/max temps
        query_table = '''
            SELECT year,
                   MIN(temperature_c) AS min_temp_c,
                   MAX(temperature_c) AS max_temp_c
            FROM weather_data
            GROUP BY year
            ORDER BY year
        '''
        report_df = pd.read_sql_query(query_table, conn)

        # Query view with UDF for max temp in Fahrenheit
        query_view = '''
            SELECT year,
                   c_to_f(MAX(temperature_c)) AS max_temp_f
            FROM positive_temps
            GROUP BY year
            ORDER BY year
        '''
        view_df = pd.read_sql_query(query_view, conn)

        # Merge results
        result = report_df.merge(view_df, on='year', how='left')
        result['max_temp_f'] = result['max_temp_f'].round(1)
        result['min_temp_c'] = result['min_temp_c'].round(1)
        result['max_temp_c'] = result['max_temp_c'].round(1)

    return result

# Step 5: Run the POC
if __name__ == "__main__":
    print("Setting up Hive-like environment...")
    setup_hive_like_db()

    print("\nGenerating Weather Temperature Statistics Report...")
    report = generate_weather_report()

    print("\n=== Weather Report ===")
    print("Year\tMin Temp (°C)\tMax Temp (°C)\tMax Temp (°F)")
    print("-" * 50)
    for _, row in report.iterrows():
        print(f"{int(row['year'])}\t{row['min_temp_c']}\t\t{row['max_temp_c']}\t\t{row['max_temp_f']}")

    print("\nSample data from view (first 5 rows):")
    with sqlite_connection('weather_hive.db') as conn:
        sample_view = pd.read_sql_query('SELECT * FROM positive_temps LIMIT 5', conn)
        print(sample_view)

Setting up Hive-like environment...
Database 'weather_hive.db', table 'weather_data', index 'idx_year', and view 'positive_temps' created.

Generating Weather Temperature Statistics Report...
UDF 'c_to_f' registered.

=== Weather Report ===
Year	Min Temp (°C)	Max Temp (°C)	Max Temp (°F)
--------------------------------------------------
1900	-43.0		27.8		82.1
1901	-43.0		41.8		107.2
1902	-47.9		48.2		118.7
1903	-45.5		44.8		112.7
1904	-45.6		32.9		91.3
1905	-10.8		23.1		73.6
1906	-46.4		36.3		97.4
1907	-34.3		47.3		117.2
1908	-26.8		48.2		118.8
1909	-46.8		29.9		85.7
1910	-37.0		33.2		91.7
1911	-48.6		42.5		108.5
1912	-49.3		27.1		80.8
1913	-44.2		35.7		96.2
1914	-31.4		48.9		120.0
1915	-36.5		19.1		66.4
1916	-48.6		-9.0		nan
1917	-44.0		42.8		109.0
1918	-47.9		48.0		118.4
1919	-39.7		30.2		86.4
1920	-24.0		47.8		118.0
1921	-37.9		25.3		77.6
1922	-44.5		17.8		64.0
1923	-49.4		49.6		121.2
1924	-45.1		24.0		75.3
1925	-45.0		25.9		78.6
1926	-33.5		46.2		115.2
1927	-36.8		7.9		46.3
1928	-1